In [153]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [154]:
# Adjusting path
# /train_data.xml
%cd "/content/drive/MyDrive/ColabNotebooks/"


/content/drive/MyDrive/ColabNotebooks


### Data Extraction from XML

In [155]:
# Loading the dataset now
import xml.etree.ElementTree as ET
tree = ET.parse('train_data.xml')
reviews = tree.getroot()


In [156]:
reviewsentencearray=[]
aspectarray=[]
polarityarray=[]
ridarray = []

i=-1

numsentences=0
for review in reviews:
  i+=1
  if(i==299):
    ridarray.append(i)
    polarityarray.append(0.0)
    aspectarray.append('FOOD#QUALITY')
    reviewsentencearray.append("Food is very bad")

  
  sentences = review.find("./sentences")
  
  for sentence in sentences:
    numsentences+=1
    
    text = sentence.find("./text")

    opinions = sentence.findall("./Opinions/Opinion")
    if(opinions == [] or opinions == None ):
      continue
    for opinion in opinions:
    
      ridarray.append(i)
      ##polarityarray.append(opinion.attrib['polarity'])
      if(opinion.attrib['polarity'] == "negative"):
        polarityarray.append(0.0)
      elif(opinion.attrib['polarity'] == "neutral"):
        polarityarray.append(0.5)
      elif(opinion.attrib['polarity'] == "positive"):
        polarityarray.append(1.0)

      aspectarray.append(opinion.attrib['category'])
      reviewsentencearray.append(text.text)


    
  
# print(numsentences)  
# print(len(reviewsentencearray))
# print(len(aspectarray))
# print(len(polarityarray))
# print(len(ridarray))

revIndex=[]
prev =0
i=0
for rid  in ridarray:
  if(prev == rid):
    i+=1
  else:
    revIndex.append(i)
    i=1
    prev=rid

maxi = -1
index =0
for i in range(len(revIndex)):
  if(revIndex[i] > maxi):
    maxi = revIndex[i]
    index = i

print("Total number of sentences are " + str(len(reviewsentencearray)))
print("Maximim number of sentences in a review is " + str(maxi))


Total number of sentences are 2508
Maximim number of sentences in a review is 44


In [157]:
if( not (len(reviewsentencearray) == len(polarityarray) ==len(aspectarray))):
  print("gone wrong some where")


Till now we have extracted Info from XML, From now on let us create a dataframe out of them right now

### introducing pandas

In [158]:

#Import pandas library
import pandas as pd

  
result=[]
for j in range(len(reviewsentencearray)):

    result.append(polarityarray[j])
    
        
data =[]
# initialize list of lists
for i in range(len(reviewsentencearray)):
  data.append([ridarray[i], aspectarray[i],reviewsentencearray[i],result[i]])
  
# Create the pandas DataFrame
DATA = pd.DataFrame(data, columns = ['rid','aspect','text','polarity'])

DATA[100:150]


,rid,aspect,text,polarity
100,14,AMBIENCE#GENERAL,The ambience is pretty and nice for conversati...,1.0
101,14,RESTAURANT#MISCELLANEOUS,The ambience is pretty and nice for conversati...,1.0
102,15,LOCATION#GENERAL,If you've ever been along the river in Weehawk...,1.0
103,15,SERVICE#GENERAL,Add to that great service and great food at a ...,1.0
104,15,FOOD#QUALITY,Add to that great service and great food at a ...,1.0
105,15,FOOD#PRICES,Add to that great service and great food at a ...,1.0
106,15,FOOD#QUALITY,The lava cake dessert was incredible and I rec...,1.0
107,16,AMBIENCE#GENERAL,"Once you step into Cosette, you're miraculousl...",1.0
108,16,AMBIENCE#GENERAL,"This tiny restaurant is as cozy as it gets, wi...",1.0
109,16,FOOD#QUALITY,The food was average to above-average; the Fre...,1.0


**Done, Now let us clean the text using standard methods avaliable for US:XD**

### Cleaning Step:  Removing Links, HTML, Emoji, Punctuation

In [159]:
import re
import string


def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)


def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)

def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)

def remove_punct(text):
  table = str.maketrans("", "", string.punctuation)
  return text.translate(table)



We have to remove stop words in the sentences for making it more efficient

In [160]:

import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords


stop = set(stopwords.words("english"))


def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [161]:
# calling all the functions declared above

DATA.text = DATA.text.map(lambda x: remove_URL(x))
DATA.text = DATA.text.map(lambda x: remove_html(x))
DATA.text = DATA.text.map(lambda x: remove_emoji(x))
DATA.text = DATA.text.map(lambda x: remove_punct(x))
DATA.text = DATA.text.map(remove_stopwords)
DATA

,rid,aspect,text,polarity
0,0,RESTAURANT#GENERAL,judging previous posts used good place longer,0.0
1,0,SERVICE#GENERAL,four us arrived noon place empty staff acted l...,0.0
2,0,SERVICE#GENERAL,never brought us complimentary noodles ignored...,0.0
3,0,FOOD#QUALITY,food lousy sweet salty portions tiny,0.0
4,0,FOOD#STYLE_OPTIONS,food lousy sweet salty portions tiny,0.0
...,...,...,...,...
2503,349,SERVICE#GENERAL,waitress came check us every minutes began cle...,0.0
2504,349,SERVICE#GENERAL,couldnt ignore fact reach plate one friends mi...,0.0
2505,349,SERVICE#GENERAL,put check without asking done came check bill ...,0.0
2506,349,RESTAURANT#GENERAL,wish could like place wish someone would retra...,0.0


**Recap of What we have done till now:**


1. Extracted all the relevant information from the xml file
2. Created a pandas dataframe out of the required data for making our lives simpler
3. Cleaned the data from pincuations, links, emojis, html   *italicized text*

**Now let us create word embeddings for proceeding with the implementaion**



### Tokenizing, Padding, Preparing Deliverables for embedding Matrix 

In [162]:
import sys
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install tensorflow

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import nltk

nltk.download('punkt')
from nltk.tokenize import word_tokenize

def create_corpus_tk(df):
    corpus = []
    for text in df["text"]:
        words = [word.lower() for word in word_tokenize(text)]
        corpus.append(words)
    return corpus



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [163]:
corpus = create_corpus_tk(DATA)

print("corpus[i] looks like:")
print(corpus[0])
print("---------------------------------")

maxi = -1
for i in range(len(reviewsentencearray)):
  if (len(corpus[i]) > maxi):
    maxi = len(corpus[i])

print("maximum words in a sentence are " + str(maxi))


corpus[i] looks like:
['judging', 'previous', 'posts', 'used', 'good', 'place', 'longer']
---------------------------------
maximum words in a sentence are 35


In [164]:
tokenizer = Tokenizer(num_words=1000000)
tokenizer.fit_on_texts(DATA.text)
word_index = tokenizer.word_index
print("number of unique words just for fun:", len(word_index))

number of unique words just for fun: 3052


Now let us create train sequences which can be processed with embeddings for catching the relation better

**IMPORTANT OBSERVATIONS:<br>
1) MAXIMUM NUMBER OF WORDS IN A SENTENCE IS : 35 <br>
2) MAXIMUM NUMBER OF SENTENCES IN A REVIEW IS : 44<br>
3) TOTAL NUMBER OF REVIEWS : 350**

In [165]:
sentences = DATA.text
sequences = tokenizer.texts_to_sequences(DATA.text)
paddedsequences = pad_sequences(
sequences, maxlen=35, truncating="post", padding="post"
)
polarity_arr = np.array(DATA.polarity)
print(paddedsequences)
print(type(paddedsequences[0]))

[[2058  609 2059 ...    0    0    0]
 [ 202   34  516 ...    0    0    0]
 [  17  611   34 ...    0    0    0]
 ...
 [ 605  347  292 ...    0    0    0]
 [ 249   91    7 ...    0    0    0]
 [ 249   91    7 ...    0    0    0]]
<class 'numpy.ndarray'>


In [166]:
newpaddingsequences = []
newpolarity = []
newaspect= []
newrid=[]
temp=[]
import numpy as np
padding = np.zeros((35))

j=0
for i in range(349):

  limit44 = 0
  while(ridarray[j]==i):
    newpaddingsequences.append(paddedsequences[j])
    newpolarity.append(polarityarray[j])
    newaspect.append(aspectarray[j])
    newrid.append(i)

    limit44+=1
    j+=1
    #print(j)
    #print(i)
  while(limit44 < 44):
    newpaddingsequences.append(padding)
    newpolarity.append(0.5)
    newaspect.append("FOOD#QUALITY")
    newrid.append(i)
    limit44 +=1


newpaddingsequences = np.array(newpaddingsequences)
newpolarity = np.array(newpolarity)
newaspect = np.array(newaspect)
newrid = np.array(newrid)
temp = np.array(temp)

  

data = []
for i in range(len(newpaddingsequences)):
  data.append([newrid[i], newaspect[i],newpaddingsequences[i],newpolarity[i]])
  
# Create the pandas DataFrame
DATA = pd.DataFrame(data, columns = ['rid','aspect','sequence','polarity'])






This sequencing and stuff is done, now let us go to word embedddings baby

In [167]:
train_size=2009

final_train_sequences = paddedsequences[:train_size]
print(final_train_sequences.shape)
# print(type(final_train_sequences[4356][0]))
# final_train_aspect = newaspect[:train_size]
final_train_labels = polarity_arr[:train_size]
# for i in range(len(final_train_labels)):
#     if(final_train_labels[i]=='negative'):
#         print(i)

final_test_sequences = paddedsequences[train_size:]
# final_test_aspect = newaspect[train_size:]
final_test_labels = polarity_arr[train_size:]

print(final_test_sequences)

(2009, 35)
[[  37    1    0 ...    0    0    0]
 [2821  647    0 ...    0    0    0]
 [   2    1    0 ...    0    0    0]
 ...
 [ 605  347  292 ...    0    0    0]
 [ 249   91    7 ...    0    0    0]
 [ 249   91    7 ...    0    0    0]]


### Embeddings

In [168]:
# import numpy as np
# embedding_dict = {}
# with open("glove.twitter.27B.100d.txt", "r") as f:
#     for line in f:
#         values = line.split()
#         word = values[0]
#         vectors = np.asarray(values[1:], "float32")
#         embedding_dict[word] = vectors
# f.close()

In [169]:
# embedding_dict

In [170]:
# num_words = len(word_index) +1
# embedding_matrix = np.zeros((num_words, 100))

# for word, i in word_index.items():
#     if i < num_words:
#         emb_vec = embedding_dict.get(word)
#         if emb_vec is not None:
#             embedding_matrix[i] = emb_vec

In [171]:
# print(len(embedding_matrix))
# len(embedding_matrix[2])

In [172]:
# print([word_index["reason"]])
# print(embedding_matrix[word_index["reason"]])


In [173]:
# print(len(embedding_dict.get("reason")))
# embedding_dict.get("reason")

In [174]:
# print(len(embedding_dict.get("food")))

In [175]:
# (embedding_matrix[1018] == embedding_dict.get("reason")).all()

In [176]:
# print(train_padded.shape)
# print(train_sentences.shape)

In [177]:
# from keras.models import Sequential
# from keras.layers import Embedding, LSTM, Dense, Dropout
# from keras.initializers import Constant
# from tensorflow.keras.optimizers import Adam


# model = Sequential()

# model.add(
#     Embedding(
#         num_words,
#         100,
#         embeddings_initializer=Constant(embedding_matrix),
#         input_length=max_len,
#         trainable=False,
#     )
# )
# model.add(LSTM(100, dropout=0.1))
# model.add(Dense(1, activation="sigmoid"))


# optimizer = Adam(learning_rate=3e-4)

# model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [178]:
#design a RNN model (write your code)

from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, SimpleRNN

out_dim = 1

model = Sequential()
model.add( Embedding(30000,out_dim,input_length = 35))#embeddings dimension as 32
model.add(SimpleRNN(16,input_shape = (2009, 35), return_sequences=False,activation="relu"))
model.add(Dense(1, activation='sigmoid'))#Dense layer of 10 units

In [179]:
from keras import losses

model.compile(loss='binary_crossentropy', 
             optimizer='rmsprop', 
             metrics=['accuracy'])

In [180]:
history = model.fit(final_train_sequences,final_train_labels,validation_data = (final_test_sequences,final_test_labels),epochs = 5 ,batch_size= 128,verbose = 1)

Epoch 1/5
16/16 [==============================] - 2s 28ms/step - loss: 0.6866 - accuracy: 0.6879 - val_loss: 0.6965 - val_accuracy: 0.4208
Epoch 2/5
16/16 [==============================] - 0s 11ms/step - loss: 0.6748 - accuracy: 0.7203 - val_loss: 0.7002 - val_accuracy: 0.4208
Epoch 3/5
16/16 [==============================] - 0s 12ms/step - loss: 0.6619 - accuracy: 0.7203 - val_loss: 0.7072 - val_accuracy: 0.4208
Epoch 4/5
16/16 [==============================] - 0s 12ms/step - loss: 0.6402 - accuracy: 0.7203 - val_loss: 0.7313 - val_accuracy: 0.4208
Epoch 5/5
16/16 [==============================] - 0s 12ms/step - loss: 0.5953 - accuracy: 0.7203 - val_loss: 0.8603 - val_accuracy: 0.4208


In [181]:
pred = model.predict(final_test_sequences)
print(pred.size)
# (model.evaluate( train_padded,
#     train_label))
print(history.history['accuracy'])
print(pred)
pred_int = pred.round().astype("int")

499
[0.687904417514801, 0.7202588319778442, 0.7202588319778442, 0.7202588319778442, 0.7202588319778442]
[[0.7291824 ]
 [0.729218  ]
 [0.7291987 ]
 [0.72880465]
 [0.7287091 ]
 [0.7287091 ]
 [0.7286601 ]
 [0.7283331 ]
 [0.7283331 ]
 [0.7289649 ]
 [0.72876954]
 [0.72876954]
 [0.7284653 ]
 [0.7284653 ]
 [0.7284653 ]
 [0.728232  ]
 [0.72904706]
 [0.7291293 ]
 [0.7288033 ]
 [0.7277001 ]
 [0.7277001 ]
 [0.7277001 ]
 [0.7288496 ]
 [0.72898173]
 [0.7291182 ]
 [0.7291516 ]
 [0.72900236]
 [0.7286327 ]
 [0.7285526 ]
 [0.7285526 ]
 [0.7285526 ]
 [0.7291055 ]
 [0.72893703]
 [0.72893703]
 [0.72893703]
 [0.72893703]
 [0.7291708 ]
 [0.7291702 ]
 [0.729107  ]
 [0.7290689 ]
 [0.7291783 ]
 [0.7285049 ]
 [0.7280698 ]
 [0.7280698 ]
 [0.7291685 ]
 [0.72913796]
 [0.7288444 ]
 [0.7288444 ]
 [0.72887594]
 [0.72887594]
 [0.7290207 ]
 [0.7291987 ]
 [0.729037  ]
 [0.72912884]
 [0.72912884]
 [0.72902477]
 [0.7286775 ]
 [0.7289666 ]
 [0.7289666 ]
 [0.72905034]
 [0.729167  ]
 [0.72885466]
 [0.7290985 ]
 [0.7290985 ]


In [182]:
print(len(pred_int))
print(len(final_test_labels))
ccc=0
for i in range(len(final_test_labels)):
    if(pred_int[i]==final_test_labels[i]):
        ccc+=1
print(ccc)

499
499
210


In [183]:
print(len(ridarray))
print(ridarray[2200:])
print(len(polarity_arr))

2508
[319, 319, 319, 319, 319, 319, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 321, 321, 321, 321, 321, 321, 321, 322, 322, 322, 322, 322, 322, 322, 322, 322, 322, 322, 322, 322, 322, 322, 323, 323, 323, 323, 323, 323, 323, 323, 324, 324, 324, 324, 324, 324, 324, 324, 324, 325, 325, 325, 325, 325, 325, 325, 325, 325, 325, 325, 325, 325, 325, 325, 325, 325, 325, 325, 325, 325, 325, 326, 326, 326, 326, 326, 326, 326, 326, 326, 327, 327, 327, 327, 327, 327, 327, 327, 327, 327, 328, 328, 328, 329, 329, 329, 330, 330, 330, 330, 330, 330, 330, 331, 331, 331, 331, 331, 331, 331, 331, 332, 332, 332, 332, 332, 333, 333, 333, 333, 333, 333, 333, 333, 333, 334, 334, 334, 334, 334, 334, 334, 334, 334, 334, 334, 334, 334, 334, 334, 334, 334, 334, 334, 335, 335, 335, 335, 335, 335, 335, 335, 335, 335, 335, 335, 335, 335, 335, 335, 335, 335, 335, 335, 335, 335, 335, 335, 335, 336, 336, 336, 336, 336, 337, 337, 337, 337, 337, 337, 337, 337, 337, 337, 337, 337, 337, 337, 337,

In [187]:
ans,idx,cnt = 0,0,0 

ans_arr = []
for i in range(len(pred)): 
  if idx == ridarray[2009+i]:
    ans+= pred_int[i]
    cnt+=1
  else:
    idx = ridarray[2009+i] 
    if cnt == 0 :
      continue
    ans_arr.append(float(ans/ cnt))
    ans = 0 
    cnt = 0

print(ans_arr)
print(len(ans_arr))

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
44


In [185]:
ans,idx,cnt = 0,0,0 

ans_polar = []
for i in range(len(pred)): 
  if idx == ridarray[2009+i]:
    ans+= polarity_arr[2009+i]
    cnt+=1
  else:
    idx = ridarray[2009+i] 
    if cnt == 0 :
      continue
    ans_polar.append(round(ans/ cnt))
    ans = 0 
    cnt = 0

print(ans_polar)
print(len(ans_polar))

[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0]
44
